In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_groq import ChatGroq

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
KEY=os.getenv("GROQ_API_KEY")

In [ ]:
# llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [5]:
llm = ChatGroq(groq_api_key=KEY,model="Gemma2-9b-It",temperature=0.5)

In [6]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002815407E590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002815407D600>, model_name='Gemma2-9b-It', temperature=0.5, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
# from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [10]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_21588\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [13]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone","response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [22]:
import json

file_path = r"D:\placement\Data Science\Generative AI\INeuron\MY\Project\1.AutoQuizzer – AI-driven MCQ generator\AutoQuizzer\Response.json"

with open(file_path, "r", encoding="utf-8") as f:
    RESPONSE_JSON = json.load(f)
    print(RESPONSE_JSON)


{'1': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [ ]:
file_path=r"D:\placement\Data Science\Generative AI\INeuron\MY\Project\1.AutoQuizzer – AI-driven MCQ generator\AutoQuizzer\data.txt"